In [1]:
import re
import shutil

import cv2
import imagesize
from src.std_imports import os, pd, np
from skmultilearn.model_selection import iterative_train_test_split

from src import annotations_processing as ap
from src import image_processing as ip
from src import constants as c
from train import multi_hot_labels



pascal_df: pd.DataFrame = ap.xml_annotations_to_dataframe(r'kaggle\input\codebrim-original\original_dataset\annotations')

original_img_dir = r'kaggle\input\codebrim-original\original_dataset\images'

(5261, 4)
(5261, 6)
(5261, 4)
(5261, 6)


In [2]:
pascal_df[pascal_df['img'] == 'image_0000005.jpg']

,img,img_width,img_height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain
0,image_0000005.jpg,1904,2856,661,472,992,1857,0,0,0,1,0,1
1,image_0000005.jpg,1904,2856,1507,505,1904,2856,0,0,0,1,0,1


In [3]:
import pandas as pd
import os

def fill_missing_imgs_in_df(img_folder, input_df):
    # Get list of images
    img_list = []
    for dirpath, dirnames, filenames in os.walk(img_folder):
        if dirpath == img_folder:
            img_list = filenames
            break
    input_df_images = input_df['img'].tolist()
    # Process images sequentially
    results = []
    for img in img_list:
        if img not in input_df_images:
            img_path = os.path.join(os.getcwd(), img_folder, img)
            # using awesome imagesize lib! Super fast! super cool!
            # https://github.com/shibukawa/imagesize_py
            shape = imagesize.get(img_path)
            ll = [0] * len(c.pascal_cols_list)
            ll[0] = img
            ll[1] = shape[0]
            ll[2] = shape[1]
            ll[7] = 1
            results.append(ll)

    # Filter out None results and add to DataFrame
    out_df = 0
    if results:
        out_df = pd.concat([input_df, pd.DataFrame(results, columns=c.pascal_cols_list)], ignore_index=True)
    return out_df

# Usage
pascal_df = fill_missing_imgs_in_df(original_img_dir, pascal_df)

In [4]:
pascal_df.sort_values(by=['img'], ascending=True, inplace=True)
pascal_df

,img,img_width,img_height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain
5261,image_0000001.jpg,1712,2572,0,0,0,0,1,0,0,0,0,0
5262,image_0000002.jpg,1732,2596,0,0,0,0,1,0,0,0,0,0
5263,image_0000003.jpg,1732,2596,0,0,0,0,1,0,0,0,0,0
5264,image_0000004.jpg,1688,2532,0,0,0,0,1,0,0,0,0,0
0,image_0000005.jpg,1904,2856,661,472,992,1857,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5257,image_0001599.jpg,4608,3456,1921,1,3105,800,0,0,1,0,1,1
5258,image_0001599.jpg,4608,3456,226,1,647,2710,0,0,0,0,0,1
5259,image_0001599.jpg,4608,3456,1002,1097,2931,3456,0,0,0,1,0,1
5253,image_0001599.jpg,4608,3456,3339,734,3736,2826,0,0,0,0,0,1


In [5]:
pascal_df[pascal_df['xmax'] > pascal_df['img_width']]

,img,img_width,img_height,xmin,ymin,xmax,ymax,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain


In [6]:
yolo_df = ap.pascal_df_to_yolo(pascal_df)
yolo_df[yolo_df['y_center'] > 1]

,img,img_width,img_height,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain,multihot_encoding_class,x_center,y_center,bbox_width,bbox_height


In [11]:
yolo_labels_dir = os.path.join(os.getcwd(), 'yolo_labels')
# if len(os.path.join(os.getcwd(), 'yolo_labels')) == 0:
ap.save_yolo_annotations(yolo_df, yolo_labels_dir)

Saved 1590 annotations to D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\yolo_labels


In [8]:
train_test_val = ap.train_test_val_image_split(yolo_df, test_size=0.2, val_size=0.1)
train_test_val['train'][0]

'image_0000001.jpg'

In [9]:
# import os
#
# # Use raw strings or forward slashes to avoid escape sequence warnings
# i1 = os.path.join(os.getcwd(), r'kaggle/input/codebrim-original/original_dataset/images')
# i2 = os.path.join(os.getcwd(), r'images/images_resized')
#
# if not os.path.exists(i2):
#     #print(os.listdir(i2))
#     os.makedirs(i2)
#
# if len(os.listdir(i2)) != 0:
#     ip.resize_images_pytorch(input_dir=i1, output_dir=i2,
#                             target_width=640, target_height=640, delete_output_dir=True)

In [12]:
def put_imgs_in_folders(input_dict: dict[str, np.ndarray], input_dir: str, base_out_dir: str) -> None:
    list_of_out_dirs = []
    for k in input_dict.keys():
        out_dir = os.path.join(base_out_dir, k)
        list_of_out_dirs.append(out_dir)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        elif len(os.listdir(out_dir)) != 0:
            raise ValueError('Output directory {} already exists'.format(out_dir))

        for img_name in input_dict[k]:
            print(img_name)
            img_name = img_name.split('.')[0] + '.txt'
            input_img_path = os.path.join(input_dir, img_name)
            out_img_path = os.path.join(out_dir, img_name)
            shutil.copy(input_img_path, out_img_path)

        print('Copying images to {} finished!'.format(out_dir))

train_test_val_labels_dir = os.path.join(os.getcwd(), 'labels')
original_img_dir = r'kaggle\input\codebrim-original\original_dataset\images'
put_imgs_in_folders(train_test_val, yolo_labels_dir, train_test_val_labels_dir)

image_0000001.jpg
image_0000003.jpg
image_0000006.jpg
image_0000008.jpg
image_0000010.jpg
image_0000012.jpg
image_0000014.jpg
image_0000016.jpg
image_0000018.jpg
image_0000020.jpg
image_0000023.jpg
image_0000025.jpg
image_0000027.jpg
image_0000028.jpg
image_0000030.jpg
image_0000032.jpg
image_0000035.jpg
image_0000037.jpg
image_0000039.jpg
image_0000041.jpg
image_0000044.jpg
image_0000047.jpg
image_0000049.jpg
image_0000050.jpg
image_0000052.jpg
image_0000053.jpg
image_0000056.jpg
image_0000058.jpg
image_0000060.jpg
image_0000062.jpg
image_0000064.jpg
image_0000066.jpg
image_0000068.jpg
image_0000070.jpg
image_0000072.jpg
image_0000075.jpg
image_0000077.jpg
image_0000078.jpg
image_0000080.jpg
image_0000082.jpg
image_0000088.jpg
image_0000090.jpg
image_0000092.jpg
image_0000093.jpg
image_0000095.jpg
image_0000098.jpg
image_0000100.jpg
image_0000102.jpg
image_0000104.jpg
image_0000106.jpg
image_0000108.jpg
image_0000109.jpg
image_0000110.jpg
image_0000112.jpg
image_0000114.jpg
image_0000

In [3]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")
results = model.train(data="yolo.yaml", epochs=100, imgsz=640, device='0', batch=8)

New https://pypi.org/project/ultralytics/8.3.61 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.56  Python-3.12.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=yolo.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\images\labels\train.cache... 170 images, 0 backgrounds, 0 corrupt: 100%|██████████| 170/170 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



C:\Users\Hubert\AppData\Roaming\Python\Python312\site-packages\ultralytics\data\augment.py:1853: UserWarning: Argument 'quality_lower' is not valid and will be ignored.
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning D:\0-Code\PG\2_sem\0_Dyplom\ai-capstone-proj\datasets\images\labels\val.cache... 954 images, 0 backgrounds, 0 corrupt: 100%|██████████| 954/954 [00:00<?, ?it/s]


Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000147, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train10
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      3/100       2.2G      2.605      6.316      2.311          2        640: 100%|██████████| 22/22 [00:02<00:00,  8.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [00:05<00:00, 11.08it/s]

                   all        954       3757          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      4/100      2.17G      2.476      6.035      2.211          3        640: 100%|██████████| 22/22 [00:02<00:00, 10.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  43%|████▎     | 26/60 [00:02<00:03,  9.47it/s]


KeyboardInterrupt: 

In [1]:
import torch

torch.cuda.is_available()

True